<a href="https://colab.research.google.com/github/Srishtiv2001/capstone/blob/codes/QUANTUM_FINALE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from typing import Tuple


def pre_computed_hamiltonian_cycles(n: int) -> list[list[Tuple]]:
    if n == 2:
        # 2 cities
        return [[(0, 1), (1, 0)]]  

    if n == 3:
        # 3 cities

        return [[(0, 1), (1, 2), (2, 0)],  
                [(0, 2), (1, 0), (2, 1)]]  

    if n == 4:
        # 4 cities

        return [[(0, 1), (1, 2), (2, 3), (3, 0)],  
                [(0, 1), (1, 3), (3, 2), (2, 0)],  
                [(0, 2), (2, 1), (1, 3), (3, 0)],  
                [(0, 2), (2, 3), (3, 1), (1, 0)],  
                [(0, 3), (3, 1), (1, 2), (2, 0)],  
                [(0, 3), (3, 2), (2, 1), (1, 0)]]  

    else:
        raise NotImplementedError

In [ ]:
import numpy as np

def find_cycle_costs(A: list[list[int]]) -> list:

    try:
        A = np.asarray(A)
    except BaseException as e:
        raise Exception("Error: Unable to convert input A to array. " + str(e))

    # Checking to make sure A is square.
    if A.shape[0] != A.shape[1]:
        raise Exception("Error: Cost matrix A must be square!")

    # Checking to make sure there are no negative costs.
    if np.any(A < 0.0):
        raise Exception("Error: The cost matrix A cannot contain negative values.")

    # Getting the maximum finite value.
    max_cost = np.amax(A[(A != np.inf) & (A != np.nan)])
    n = len(A)  # This is the number of cities in our problem.

    # Removing all infinities and nans.
    A = np.nan_to_num(A, copy=True, nan=0.0, posinf=n * max_cost)

    # Using the precomputed Hamiltonian cycles to compute the costs.
    hamiltonian_cycles = pre_computed_hamiltonian_cycles(n=n)
    number_of_cycles = len(hamiltonian_cycles)

    cycle_costs = np.full(shape=number_of_cycles, fill_value=0.0)  # Pre-allocate
    for i, cycle in enumerate(hamiltonian_cycles):
        # Each cycle is a list of tuples.
        for t in cycle:
            cycle_costs[i] += A[t[0], t[1]]

    return list(cycle_costs.astype(int))


if __name__ == "__main__":

    print("### 3 city example (n = 3) ###")
    A3 = [[3, 7, 2],
          [5, 12, 9],
          [17, 1, np.inf]]

    print("Cost matrix A:")
    print(A3)

    print("\nCycle costs:")
    costs = find_cycle_costs(A=A3)
    print(costs)

    print("\n### 4 city Example (n = 4) ### ")
    A4 = [[3, 7, 2, 19],
          [5, 12, np.nan, 5],
          [17, 1, np.inf, 2.4],
          [3, 13, 13, 12]]

    print("Cost matrix A:")
    print(A4)

    print("\nCycle costs:")
    costs = find_cycle_costs(A=A4)
    print(costs)

    # Poster Example
    print("\n###  Poster Example (n = 4) ###")
    A = [[0, 7, 2, 7],
         [5, 0, 3, 5],
         [2, 1, 0, 8],
         [3, 13, 6, 0]]

    print("Cost matrix A:")
    print(A)

    print("\nCycle costs:")
    costs = find_cycle_costs(A=A)
    print(costs)

### 3 city example (n = 3) ###
Cost matrix A:
[[3, 7, 2], [5, 12, 9], [17, 1, inf]]

Cycle costs:
[33, 8]

### 4 city Example (n = 4) ### 
Cost matrix A:
[[3, 7, 2, 19], [5, 12, nan, 5], [17, 1, inf, 2.4], [3, 13, 13, 12]]

Cycle costs:
[12, 42, 11, 22, 49, 38]

###  Poster Example (n = 4) ###
Cost matrix A:
[[0, 7, 2, 7], [5, 0, 3, 5], [2, 1, 0, 8], [3, 13, 6, 0]]

Cycle costs:
[21, 20, 11, 28, 25, 19]


In [ ]:
import math

import pennylane as qml
import matplotlib.pyplot as plt

from numpy import pi as pi


def minimization_oracle(data_register: list[int], ancillary_register: list[int], arr: list[int], x: int,
                        phi: float) -> None:

    def add_k_fourier(k: int, wires: list[int]) -> None:
        #Add the integer value k to wires.
        for j in range(len(wires)):
            qml.RZ(k * pi / (2 ** j), wires=wires[j])

    def load_values_in_arr():
        #Conditionally load all the values in arr onto the axcillary register.
        # Performing a QFT, so we can add values in the Fourier basis.
        qml.QFT(wires=ancillary_register)

        # Looping through each of the data wires, performing controlled additions.
        for wire in data_register:
            qml.ctrl(op=add_k_fourier, control=wire)(k=arr[wire], wires=ancillary_register)

        # Returning to the computational basis.
        qml.adjoint(fn=qml.QFT(wires=ancillary_register))

    load_values_in_arr()

    # Loop thorugh all integers in the range 1 to x-1. Notice we start at 1 because the all 0's state would always be
    #  marked. Anyway, in the TSP context arr should never contain any 0's. If we get a sum of elements on the
    #  ancillary wires that is < x, then there must be at least one element in arr that is less than x.
    for i in range(1, x):

        # qml.FlipSign(i, wires=ancillary_register)
        # # To improve sucess, we replace the phase inversion by a phase rotation through phi.

        # Implement flip_sign in pauli primatives
        arr_bin = to_list(n=i, n_wires=len(ancillary_register))  # Turn i into a state.

        if arr_bin[-1] == 0:
            qml.PauliX(wires=ancillary_register[-1])

        # qml.ctrl(qml.PauliZ, control=ancillary_register[:-1], control_values=arr_bin[:-1])(wires=ancillary_register[-1])

        # Perform with RZ instead of Pauli-Z so we can do an arbitrary angle.
        if arr_bin[-1] == 0:
            qml.RZ(phi=phi, wires=ancillary_register[-1])
        qml.ctrl(qml.PauliX, control=ancillary_register[:-1], control_values=arr_bin[:-1])(
            wires=ancillary_register[-1])
        if arr_bin[-1] == 0:
            qml.RZ(phi=-phi, wires=ancillary_register[-1])
        qml.ctrl(qml.PauliX, control=ancillary_register[:-1], control_values=arr_bin[:-1])(
            wires=ancillary_register[-1])

        if arr_bin[-1] == 0:
            qml.PauliX(wires=ancillary_register[-1])

    qml.adjoint(fn=load_values_in_arr)()  # Cleanup.


def to_list(n, n_wires):
    # From PennyLane
    if n >= 2 ** n_wires:
        raise ValueError(f"cannot encode {n} with {n_wires} wires ")

    b_str = f"{n:b}".zfill(n_wires)
    bin_list = [int(i) for i in b_str]
    return bin_list


def minimization_circuit(data_register: list[int], ancillary_register: list[int], arr: list[int], x: int,
                         ret: str = "sample") -> list:

    # Step 1: Create an equal superposition by applying a Hadamard to each wire in the data register.
    for wire in data_register:
        qml.Hadamard(wires=wire)

    # Compute some of the parameters defined in Long et al.
    beta = math.asin(1 / math.sqrt(len(arr)))
    j_op = math.floor((pi / 2 - beta) / (2 * beta))
    j = j_op + 1
    phi = 2 * math.asin(math.sin(pi / (4 * j + 6)) / math.sin(beta))

    # print("phi: " + str(phi))
    # print("j_op: " + str(j_op))
    # print("j: " + str(j))
    # print("beta: " + str(beta))

    # Upon measurment at the (J + 1)th iteration, the marked state is obtained with quasi-certainty.
    for _ in range(j + 1):
        # Step 2: Use the oracle to mark solution states.
        minimization_oracle(data_register=data_register, ancillary_register=ancillary_register, arr=arr, x=x, phi=pi)

        # Step 3: Apply the Grover operator to amplify the probability of getting the correct solution.
        qml.GroverOperator(wires=data_register)

    if ret == "probs":
        return qml.probs(wires=data_register)
    else:
        return qml.sample(wires=data_register)


def grover_for_minimization(arr: list[int], x: int) -> bool:
    # We need one wire in the data register for each element of arr.
    number_of_data_qubits_required = len(arr)

    # We need enough ancillary qubits to represent 0 -> sum(arr) or x, whichever is larger.
    #  Recall you can represent up to 2^n - 1 with n bits.
    number_ancillary_qubits_required = max(math.ceil(math.log(sum(arr) + 1, 2)), math.ceil(math.log(x + 1, 2)))

    data_register = list(range(0, number_of_data_qubits_required))  # One data qubit for each element of arr
    ancillary_register = list(range(number_of_data_qubits_required,
                                    number_of_data_qubits_required + number_ancillary_qubits_required))
    print(len(data_register))
    print(len(ancillary_register))

    # We will create 2 devices, one first for creating a plot of the probabilities.
    device1 = qml.device("default.qubit", wires=data_register + ancillary_register)
    qnode1 = qml.qnode(func=minimization_circuit, device=device1)(
        data_register=data_register, ancillary_register=ancillary_register, arr=arr, x=x, ret="probs")

    # Obtain and plot a probablity distribution.
    values = qnode1.__call__(data_register=data_register, ancillary_register=ancillary_register, arr=arr, x=x,
                             ret="probs")
    print("\nlen(values):")  # Four bits can encode 16 distinct characters
    print(len(values))
    print("\nvalues:")
    print(values)
    # plt.rcParams["figure.figsize"] = [7.50, 6]
    # plt.rcParams["figure.autolayout"] = True
    # plt.bar(range(len(values)), values)
    # plt.xlabel("State", fontsize=25)
    # plt.ylabel("Probability", fontsize=25)
    # plt.xticks(fontsize=20)
    # plt.yticks(fontsize=20)

    # plt.show()

    # The second device is for actully obtaining a sample.
    device2 = qml.device("default.qubit", wires=data_register + ancillary_register, shots=1)
    qnode2 = qml.qnode(func=minimization_circuit, device=device2)(
        data_register=data_register, ancillary_register=ancillary_register, arr=arr, x=x, ret="sample")

    sample = qnode2.__call__(data_register=data_register, ancillary_register=ancillary_register, arr=arr, x=x,
                             ret="sample")
    print(sample)

    found_elements = [i for indx, i in enumerate(arr) if sample[indx] == 1]
    print("Found elements: " + str(found_elements))

    if len(found_elements) > 0:
        if found_elements[0] < x:
            # If the found value is actually less than result, great!
            return True

    # No such element exists.
    return False


if __name__ == "__main__":

    arr_ = [18, 10, 6, 7]
    # arr_ = [3, 5, 4, 3]
    # arr_ = [36, 5, 8, 14, 15, 2, 4, 16]
    x_ = 3

    found_number_smaller_than_x = grover_for_minimization(arr=arr_, x=x_)

    print("Found a number smaller than x: " + str(found_number_smaller_than_x))

4
6

len(values):
16

values:
[0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
[1 0 1 1]
Found elements: [18, 6, 7]
Found a number smaller than x: False


In [ ]:
def grover_enhanced_minimization(arr: list[int], _lower_bound: int = 0, _upper_bound: int = None,
                                 verbose: bool = False) -> int:
    if _upper_bound is None:
        # First iteration. For an initial upper bound, just use the first element in arr. Worst case, this is the
        #  biggest element.
        _upper_bound = arr[0]
        if _lower_bound > _upper_bound:
            raise Exception("Error: The initial_bound must be less than arr[0].")

    if verbose:
        print("\nLower bound: " + str(_lower_bound))
        print("Upper bound: " + str(_upper_bound))

    # We keep going till the upper and lower bounds cross.
    if _upper_bound >= _lower_bound:
        middle = (_upper_bound + _lower_bound) // 2

        # Using Grover, check if there is an element smaller than the middle value.
        # smaller_element_exists = grover_for_minimization_classical(arr=arr, x=middle)
        smaller_element_exists = grover_for_minimization(arr=arr, x=middle)

        if smaller_element_exists:
            # There is at least one element smaller than middle, lower our upper bound.
            if verbose:
                print("Smaller element found, lowering our upper bound...")
            return grover_enhanced_minimization(arr, _lower_bound=_lower_bound, _upper_bound=middle - 1,
                                                verbose=verbose)

        else:
            # There are no elements smaller than middle, raise the lower bound.
            if verbose:
                print("No smaller element found, raising our lower bound...")
            return grover_enhanced_minimization(arr, _lower_bound=middle + 1, _upper_bound=_upper_bound,
                                                verbose=verbose)

    else:
        # Bounds have crossed, return the solution.
        return _upper_bound


def grover_for_minimization_classical(arr: list[int], x: int) -> bool:
    for i in range(len(arr)):
        if arr[i] < x:
            return True  # We have found a smaller value!

    return False  # No element in list > middle


if __name__ == "__main__":

    # test_array = [27, 15, 17, 16, 3, 45, 9]
    test_array = list(range(0, 10))
    test_array.reverse()  # Engineering the worst case scenario - when the first element in the list is the largest.

    # print(test_array)
    grover_enhanced_minimization(arr=test_array, verbose=False)

    
    # results = [0] * 100
    # for i in range(100):
    #     results[i] = grover_enhanced_minimization(arr=test_array, verbose=True)
    #     print("###")
    #     print("MIN: " + str(results[i]))
    #     print("###")
    #
    # print(results)

10
6

len(values):
1024

values:
[0.00072581 0.00072581 0.00072581 ... 0.00072581 0.00072581 0.00072581]
[0 0 0 0 1 0 1 0 1 0]
Found elements: [5, 3, 1]
10
6

len(values):
1024

values:
[8.15511154e-05 8.15511154e-05 8.15511154e-05 ... 8.15511154e-05
 8.15511154e-05 8.15511154e-05]
[1 0 0 1 0 1 0 0 0 1]
Found elements: [9, 6, 4, 0]
10
6

len(values):
1024

values:
[8.15511154e-05 8.15511154e-05 8.15511154e-05 ... 8.15511154e-05
 8.15511154e-05 8.15511154e-05]
[0 0 0 1 0 0 0 0 1 0]
Found elements: [6, 1]


In [ ]:
import pennylane as qml
from pennylane.templates import QuantumPhaseEstimation
from pennylane import numpy as np
from typing import Tuple



def build_U(A: list[list[int]], verbose: False) -> Tuple[np.asarray, Tuple[float, float]]:
    n = len(A)  # Number of cities in the problem
    cycle_costs = np.asarray(find_cycle_costs(A=A), dtype=float)

    # Normalizing all the cycle costs so they are in the range [0, 2 * pi).
    # Used Durr and Hoyer quantum-enhanced optimation here to compute the min and max values of A.
    cycle_costs_max = n * np.amax(A)  # Largest possbily cycle cost
    cycle_costs_min = n * np.amin(A)  # Smallest possible cycle cost

    normalized_cycle_costs = cycle_costs - cycle_costs_min
    normalized_cycle_costs *= 2 * np.pi / cycle_costs_max

    diagonal_elements = np.exp(1j * normalized_cycle_costs)
    U = np.diag(v=diagonal_elements, k=0)

    if verbose:
        print("\nCycle costs: " + str(cycle_costs))

        print("Max possible cycle cost: " + str(cycle_costs_max))
        print("Min possible cycle cost: " + str(cycle_costs_min))

        print("\nCycle costs after normalization:")
        print(normalized_cycle_costs)

        print("\nDiagonal elements:")
        print(diagonal_elements)

    return U, (cycle_costs_min, cycle_costs_max)


def phase_estimation(U: list[list]):
    # The number of target wires required depends on the size of U
    n_target_wires = math.ceil(math.log(len(U), 2))

    # We get to choose the number of estimation wires to use. The error in our estimate will decrease exponentially
    #  with the number of estimation qubits.
    n_estimation_wires = 5

    thetas = np.full(shape=len(U), fill_value=np.nan)

    for i in range(len(U)):
        # Perform phase estimation to solve U|i> = lambda |i>

        target_wires = list(range(0, n_target_wires))
        estimation_wires = range(n_target_wires, n_target_wires + n_estimation_wires)

        # Create a device, node, and execute the quntum circuit.
        device = qml.device("default.qubit", wires=n_estimation_wires + 1)
        qnode = qml.qnode(func=phase_estimation_circuit, device=device)()

        values = qnode.__call__(unitary=U, target_wires=target_wires, estimation_wires=estimation_wires, i=i)
        # print("$$$$$$$$$$$values:",values)

        # Find the index of the largest value in the probability distribution and divide that number by 2^n.
        theta = np.argmax(values) / 2 ** n_estimation_wires
        # eigenvalue = np.exp(2 * np.pi * 1j * theta)

        thetas[i] = theta

    return thetas


def phase_estimation_circuit(unitary: list[list[float]], target_wires: list[int], estimation_wires: list[int],
                             i: int) -> float:
    # Prepare target wires in the ith basis vector.
    qml.BasisState(np.asarray([i]), wires=target_wires)
    QuantumPhaseEstimation(unitary, target_wires=target_wires, estimation_wires=estimation_wires)

    return qml.probs(estimation_wires)


if __name__ == "__main__":

    print("n = 3:")
    A3 = np.asarray([[0, 7, 2],
                     [5, 0, 9],
                     [17, 1, 0]])
    


    print("\nA:")
    print(A3)

    # Using A, build a unitary matrix where the costs are encoded as phases.
    U_, cycle_cost_range_ = build_U(A=A3, verbose=True)
    print("\nUnitary matrix U:")
    print(U_)
    print("Cycle cost range: " + str(cycle_cost_range_[0]) + ", " + str(cycle_cost_range_[1]))

    # Use phase estimation to obtain the phases (phases encode total costs).
    thetas = phase_estimation(U=U_)

    # Now that we have the phases, convert them back into costs.
    cycle_costs_found = thetas * cycle_cost_range_[1]
    cycle_costs_found += cycle_cost_range_[0]

    print("\nOutput phases:")
    print(cycle_costs_found)

    print("\nn = 4:")
    A4 = np.asarray([[3, 7, 2, 19],
                     [5, 12, np.nan, 5],
                     [17, 1, np.inf, 2.4],
                     [3, 13, 13, 12]])
    
    print("\nA:")
    print(A4)

n = 3:

A:
[[ 0  7  2]
 [ 5  0  9]
 [17  1  0]]

Cycle costs: [33.  8.]
Max possible cycle cost: 51
Min possible cycle cost: 0

Cycle costs after normalization:
[4.06559049 0.9855977 ]

Diagonal elements:
[-0.60263464-0.79801723j  0.55236497+0.83360239j]

Unitary matrix U:
[[-0.60263464-0.79801723j  0.        +0.j        ]
 [ 0.        +0.j          0.55236497+0.83360239j]]
Cycle cost range: 0, 51

Output phases:
[33.46875  7.96875]

n = 4:

A:
[[ 3.   7.   2.  19. ]
 [ 5.  12.   nan  5. ]
 [17.   1.   inf  2.4]
 [ 3.  13.  13.  12. ]]


In [ ]:
import numpy as np

def tsp(A: list[list[int]]) -> int:
    # Using A, build a unitary matrix where the costs are encoded as phases.
    U, cycle_cost_range = build_U(A=A, verbose=False)

    # Use phase estimation to obtain the phases (phases encode total costs).
    thetas = phase_estimation(U=U_)

    # Now that we have the phases, convert them back into costs.
    cycle_costs_found = thetas * cycle_cost_range[1]
    cycle_costs_found += cycle_cost_range[0]

    # Right now our Grover enhanced minimization only works for interger lists. Our toal costs should be integer anyway.
    cycle_costs_rounded = [int(item) for item in cycle_costs_found]
    lowest_cost = grover_enhanced_minimization(arr=cycle_costs_rounded, verbose=False)

    return lowest_cost


if __name__ == "__main__":

    print("### 3 City Example (n = 3) ###")
    A = [[0, 7, 2],
         [5, 0, 9],
         [17, 1, 0]]

    print("\nA:")
    print(A)

    print("\nCost of shortest cost circuit:")
    print(tsp(A=A))

    print("\n### 4 city Example (n = 4) ### ")
    A = [[3, 7, 2, 19],
         [5, 12, 13, 5],
         [17, 1, 10, 2],
         [3, 13, 13, 12]]

    print("\nA:")
    print(A)

    print("\nCost of shortest cost circuit:")
    print(tsp(A=A))

### 3 City Example (n = 3) ###

A:
[[0, 7, 2], [5, 0, 9], [17, 1, 0]]

Cost of shortest cost circuit:
2
6

len(values):
4

values:
[0.25 0.25 0.25 0.25]
[1 1]
Found elements: [33, 7]
2
6

len(values):
4

values:
[0.25 0.25 0.25 0.25]
[1 1]
Found elements: [33, 7]
2
6

len(values):
4

values:
[0.25 0.25 0.25 0.25]
[1 0]
Found elements: [33]
2
6

len(values):
4

values:
[0.25 0.25 0.25 0.25]
[0 0]
Found elements: []
2
6

len(values):
4

values:
[0.25 0.25 0.25 0.25]
[0 1]
Found elements: [7]
31

### 4 city Example (n = 4) ### 

A:
[[3, 7, 2, 19], [5, 12, 13, 5], [17, 1, 10, 2], [3, 13, 13, 12]]

Cost of shortest cost circuit:
2
7

len(values):
4

values:
[0.25 0.25 0.25 0.25]
[1 1]
Found elements: [53, 15]
2
7

len(values):
4

values:
[0.25 0.25 0.25 0.25]
[0 0]
Found elements: []
2
7

len(values):
4

values:
[0.25 0.25 0.25 0.25]
[1 1]
Found elements: [53, 15]
2
7

len(values):
4

values:
[0.25 0.25 0.25 0.25]
[1 1]
Found elements: [53, 15]
2
7

len(values):
4

values:
[0.25 0.25 0.25 0